## Importando bibliotecas

In [1]:
import pandas as pd
import openpyxl

## Carregando dados

In [2]:
df = pd.read_excel('https://github.com/TrainingByPackt/Data-Science-Projects-with-Python/raw/master/Data/default_of_credit_card_clients__courseware_version_1_21_19.xls')
df.shape

(30000, 25)

## Entendendo contexto/problema

Cliente = Empresa de Cartão de Crédito

Dados históricos = 6 meses de 30 mil clientes

Objetivo = *Modelo capaz de prever se uma conta/cliente ficará inadimplente no próximo mês.*

## Verificando integridade dos dados

In [3]:
print("Total de colunas", len(df.columns), "\n")
df.columns

Total de colunas 25 



Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')

## Dicionário de dados 

### Informações do Conjunto de Dados

Esta pesquisa utilizou uma variável binária, pagamento padrão (Sim = 1, Não = 0), como variável de resposta. Este estudo revisou a literatura e utilizou as seguintes 23 variáveis como variáveis explicativas:

1. **X1: Valor do crédito concedido (em dólares taiwaneses - NT)**
   - Inclui tanto o crédito individual do consumidor quanto o crédito familiar (suplementar).

2. **X2: Gênero**
   - 1 = Masculino
   - 2 = Feminino

3. **X3: Educação**
   - 1 = Pós-graduação
   - 2 = Universidade
   - 3 = Ensino médio
   - 4 = Outros

4. **X4: Estado Civil**
   - 1 = Casado
   - 2 = Solteiro
   - 3 = Outros

5. **X5: Idade (em anos)**

6. **X6 - X11: Histórico de Pagamento Anterior**
   - X6 = Status de pagamento em setembro de 2005
   - X7 = Status de pagamento em agosto de 2005
   - ...
   - X11 = Status de pagamento em abril de 2005
   - Escala de medição: -1 = Pagar pontualmente; 1 = Atraso no pagamento por um mês; 2 = Atraso no pagamento por dois meses; ...; 8 = Atraso no pagamento por oito meses; 9 = Atraso no pagamento por nove meses ou mais.

7. **X12-X17: Valor do Extrato da Fatura (em dólares taiwaneses)**
   - X12 = Valor do extr


In [4]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


## Verificando ID

In [5]:
df["ID"].nunique()

29687

In [6]:
id_counts = df["ID"].value_counts()
id_counts.head()

ID
ad23fe5c-7b09    2
1fb3e3e6-a68d    2
89f8f447-fca8    2
7c9b7473-cc2f    2
90330d02-82d9    2
Name: count, dtype: int64

In [7]:
id_counts.value_counts()

count
1    29374
2      313
Name: count, dtype: int64

## Corrigindo dados duplicados

In [8]:
dupe_mask = id_counts == 2
dupe_mask[:5]

ID
ad23fe5c-7b09    True
1fb3e3e6-a68d    True
89f8f447-fca8    True
7c9b7473-cc2f    True
90330d02-82d9    True
Name: count, dtype: bool

In [9]:
dupe_ids = id_counts.index[dupe_mask]

In [10]:
dupe_ids = list(dupe_ids)
len(dupe_ids)

313

In [11]:
dupe_ids[:5]

['ad23fe5c-7b09',
 '1fb3e3e6-a68d',
 '89f8f447-fca8',
 '7c9b7473-cc2f',
 '90330d02-82d9']

In [16]:
df.loc[df["ID"].isin(dupe_ids[0:3]), :].head(10)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
5033,89f8f447-fca8,320000,2,2,1,32,0,0,0,0,...,169371,172868,150827,8000,8000,5500,6100,6000,5000,0
5133,89f8f447-fca8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16727,1fb3e3e6-a68d,80000,1,2,2,33,2,2,0,0,...,27394,29922,31879,0,2000,2000,3000,2600,0,1
16827,1fb3e3e6-a68d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29685,ad23fe5c-7b09,50000,1,3,1,32,0,0,0,0,...,12882,8131,3983,3000,2871,1000,163,3983,3771,1
29785,ad23fe5c-7b09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df_zero_mask = df == 0

In [22]:
feature_zero_mask = df_zero_mask.iloc[:, 1:].all(axis=1)

In [23]:
sum(feature_zero_mask)

315

In [26]:
df_clean_1 = df.loc[~feature_zero_mask,:].copy()

In [27]:
df_clean_1.shape

(29685, 25)

In [28]:
df_clean_1['ID'].nunique()

29685